In [1]:
import brickschema
from brickschema.namespaces import BRICK, RDFS, RDF
import json
from element_relationship import Element_Relationship_Extractor
import rdflib

In [2]:
with open("config-cdl.json") as fp:
    config = json.load(fp)
config


{'model': 'Buildings.Controls.OBC.ASHRAE.G36_PR1.TerminalUnits.Controller',
 'generate_json': True}

In [3]:
element_extractor = Element_Relationship_Extractor(config_file = "config-cdl.json");

In [4]:
elements, relationships = element_extractor.extract_class_definition()

In [9]:
graph = {}
bldg_graph = brickschema.Graph()
BLDG = rdflib.Namespace("urn:bldg/")
bldg_graph.bind("bldg", BLDG)
ref_ns = rdflib.Namespace("https://brickschema.org/schema/Brick/ref#")
cdl_ns = rdflib.Namespace("http://data.ashrae.org/cdl#")
bldg_graph.bind("cdl", cdl_ns)

In [11]:
parameters = []
inputs = []
outputs = []

for element in elements:
    type_specifier = elements[element].get('type_specifier', None)
    type_prefix = elements[element].get('type_prefix', None)
    if type_specifier is not None and ('RealInput' in type_specifier or 'IntegerInput' in type_specifier):
        bldg_graph.add((BLDG[element], rdflib.RDF.type, cdl_ns['AnalogInput']))
    elif type_specifier is not None and 'BooleanInput' in type_specifier:
        bldg_graph.add((BLDG[element], rdflib.RDF.type, cdl_ns['BinaryInput']))
    elif type_specifier is not None and ('RealOutput' in type_specifier or 'IntegerOutput' in type_specifier):
        bldg_graph.add((BLDG[element], rdflib.RDF.type, cdl_ns['AnalogOutput']))
    elif type_specifier is not None and 'BooleanOutput' in type_specifier:
        bldg_graph.add((BLDG[element], rdflib.RDF.type, cdl_ns['BinaryOutput']))
    elif type_prefix is not None and "parameter" in elements[element].get('type_prefix'):
        bldg_graph.add((BLDG[element], rdflib.RDF.type, cdl_ns['Parameter']))

In [13]:
bldg_graph.serialize(destination='{}_231p.ttl'.format(config.get('model')), format='turtle')

<Graph identifier=Nf3e92f8d2ef3448d8dee27ec56513e68 (<class 'brickschema.graph.Graph'>)>